Inhaltsübersicht EDA
TD = To Do, TBD = To Be Determined)

1. Überblick über den Datensatz
- Spaltenbeschreibung/Datentypen: Funktion overview()
- Fehlende Werte (NaN-Anteile berechnet, diskutiert)

- TBD: evtl. kurze Beschreibung der Daten (Ursprung, Umfang) --> ins Datenwörterbuch oder DataHowTo

2. Datenbereinigung
- Entfernen unrealistischer Ausreißer (Schadstoffe, Wetterdaten) --> Städte auch noch überprüfen?
- Zellenweise Ersetzung durch NaN (statt Zeilenlöschung)
- Dokumentaton der Schwellenwerte und Filterlogik

- TD: evtl. Tabelle oder Text mit Anzahl der entfernten/extremen Werte

3. Deskriptive Statistik (Grundanalyse)
- describe()-Tabellen für Schadstoffe und Wetterdaten
- Vergleich von Mean und Median zu Einschätzung der Schiefe
- Berechnung der Schiefe (skew)

4. Visualisierungen
- Stripplots zur Ausreißererkennung für alle numerischen Spalten
- Histogramme für ausgewählte Variablen, zur Visualisierung der Schiefe hauptsächlich, aber auch für Platzhalterwerte
- Korrelationsmatrix (Pearson) --> Dreieck

- TD: Lineplots mit Mittelwert und Streuung (für einzelnen Schadstoffe, noch ausbaufähig)
- TD: Weitere Diagramme nach Gruppen (Stadt, Jahr, Kontinent, etc.)

5. Korrelationsanalyse
- Korrelationen berechnet und interpretiert
- Stärkste Korrelationen identifiziert und dokumentiert
- Empfehlung zur Variablenselektion (evtl. PCA) gegeben

6. Prüfung auffälliger Größen/Städte (Feedback-Loop aus der Clusteranalyse; weiter oben bei Ausreißerbestimmung einbauen?)
- Geprüft wurden: Ashkelon, Ashdod, Netanya

- TD: Weitere Städte prüfen (z.B. Tehran noch mal genau berechnen)

7. Vorbereitung für Clustering und andere Modellierungen
- Welche Features sollen verwendet werden?
- Standardisierung / Skalierung wie?
- Ziel der Clusteranalyse formulieren

Allgemein: Dokumentation & Kommunikation
- Kommentare und Begründungen im Code, aber mit Markdown
- Einführung und Fazit zur EDA?
- Strukturierung der finalen NB-Version für GitHub
- TD: Einzelergebnisse abspeichern als csv




In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import os
# import matplotlib
# import datetime

In [ ]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
df = pd.read_csv("./data/cleaned_data.csv")
df.head()

In [ ]:
df.columns

# Some redundant columns still need to be removed from df manually: Temperature, Pressure, Wind-speed

In [ ]:
# Drop redundant columns

df = df.drop(columns=['Temperature', 'Wind-speed', 'Pressure'])
df.columns

In [ ]:
# Function to create overview for dtypes, missing values, unique values

def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))


In [ ]:
overview(df)

In [ ]:
# Focus on missing values: Sort missing values (descending)

missing_percent = df.isna().mean() * 100
missing_percent_sorted = missing_percent.sort_values(ascending=False)

print(missing_percent_sorted)


In [ ]:
# Remove all columns with more than 50% missing values
# Note: This removes population data entirely!

df = df.loc[:, missing_percent <= 50]

# Print what has changed
print(f"Anzahl der entfernten Spalten: {df.shape[1] - df.shape[1]}")
print("Übrige Spalten:", df.columns)

df.head()

In [ ]:
df.columns

In [ ]:
df.shape

# Deskriptive Statistik

Dataset has many variables. Statistics will be split up into pollutants, weather and location

# Analysis of data on pollutants

In [ ]:
# Pollutants

pollutant_cols = ["Co", "Pm25", "Pm10", "So2", "No2", "O3"]
df[pollutant_cols].describe().T


Einige Messwerte wie Co = 500 oder Pm25 = 999 deuten auf Platzhalter für fehlende oder ungültige Daten hin und verzerren die Verteilung erheblich. Diese Werte könnten fehlerhaft sein und sollten aus der Analyse ausgeschlossen werden.

## Analyse und Bereinigung der Pm25-Werte

In [ ]:
df["Pm25"].sort_values(ascending=False).unique() [:10]

In [ ]:
sns.stripplot(x=df["Pm25"], size=2, jitter=0.3)
plt.title("PM2.5 Einzelwerte")
plt.xlabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

### Interpretation der Pm25-Werte VOR Bereinigung des Datensatzes
Bei der visuellen Analyse der PM2.5-Messwerte fällt eine ungewöhnliche Häufung im Bereich zwischen 816 und 834 µg/m³ auf, begleitet von vereinzelten extrem hohen Werten bei 981 und 999 µg/m³. Diese Werte liegen deutlich außerhalb des realistischen Messbereichs und deuten auf Messfehler, Platzhalterwerte oder systembedingte Übertragungsfehler hin. Um die Analyse nicht zu verzerren, werden daher alle PM2.5-Werte ab 816 µg/m³ ausgeschlossen.

In [ ]:
# PM2.5 >= 816
df.loc[df["Pm25"] >= 816, "Pm25"] = np.nan

In [ ]:
sns.histplot(df["Pm25"], bins=100)

Die Verteilung der Messwerte wirkt nach dem Entfernen der unrealistischen Werte rechtsschief. Überprüfung mit Vergleich Mean/Median.

In [ ]:
mean_pm25 = df["Pm25"].mean()
median_pm25 = df["Pm25"].median()

print(f"Mittelwert (mean): {mean_pm25:.2f}")
print(f"Median:            {median_pm25:.2f}")

In [ ]:
from scipy.stats import skew

skew_pm25 = skew(df["Pm25"].dropna())
print(f"Schiefe (Skewness): {skew_pm25:.2f}")

# Skewness > 0 --> rechtsschiefe Verteilung.

### Verteilung der bereinigten PM2.5-Werte (Fazit)

Die PM2.5-Werte zeigen eine deutlich rechtsschiefe Verteilung. Der Mittelwert liegt bei 53,19 µg/m³, während der Median nur 42,00 µg/m³ beträgt. Zusätzlich ergibt die Berechnung der Schiefe (Skewness) einen Wert von 2,26, was auf eine starke Asymmetrie hindeutet.
Diese rechtsschiefe Verteilung bedeutet, dass der Großteil der Messwerte im unteren Bereich liegt, während einige wenige sehr hohe Werte den Mittelwert nach oben ziehen.

Um die Analyse nicht durch technisch bedingte Ausreißer zu verzerren, wurden alle PM2.5-Werte ab 816 µg/m³ ausgeschlossen, da sie deutlich außerhalb des natürlichen Messbereichs liegen und teilweise systematisch auftraten.

## Analyse und Bereinigung der CO-Werte

In [ ]:
df["Co"].sort_values(ascending=False).unique()[:30]

In [ ]:
sns.stripplot(x=df["Co"], size=2, jitter=0.3)
plt.title("CO Einzelwerte")
plt.xlabel("CO-Konzentration (µg/m³)")
plt.grid(True);

### Interpretation der Pm25-Werte VOR Bereinigung des Datensatzes
Es gibt zwar einen eindeutigen Platzhalterwert, aber alle anderen Messwerte könnten theoretisch realistisch sein. Überprüfung der Verteilung mit Mean/Median, um den Cutoff festzulegen.

In [ ]:
mean_co = df["Co"].mean()
median_co = df["Co"].median()
skew_co = skew(df["Co"].dropna())

print(f"Mittelwert (mean): {mean_co:.2f}")
print(f"Median:            {median_co:.2f}")
print(f"Schiefe:           {skew_co:.2f}")

### Bereinigte CO-Werte (Begründung)

Die Verteilung der CO-Messwerte ist stark rechtsschief, mit einem Mittelwert von 4.74 µg/m³, einem Median von 3.40 µg/m³ und einer Schiefe von 29.64. Dies deutet auf extreme Ausreißer hin, die den Mittelwert erheblich verzerren. Zusätzlich tritt eine auffällige Häufung exakt bei 500 µg/m³ auf, was sehr wahrscheinlich ein technischer Platzhalterwert ist.

Um die Aussagekraft der Analyse zu erhöhen, wurden daher alle Werte ab 300 µg/m³ ausgeschlossen. Dieser Grenzwert liegt deutlich oberhalb der Hauptverteilung und entfernt sowohl technische Ausreißer als auch extreme Einzelwerte.

In [ ]:
# CO > 300 oder CO == 500 → als NaN
df.loc[df["Co"] >= 300, "Co"] = np.nan

## Analyse und Bereinigung der No2-Werte

In [ ]:
sns.stripplot(x=df["No2"], size=2, jitter=0.3)
plt.title("NO₂ Einzelwerte")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
mean_no2 = df["No2"].mean()
median_no2 = df["No2"].median()
skew_no2 = skew(df["No2"].dropna())

print(f"Mittelwert (mean): {mean_no2:.2f}")
print(f"Median:            {median_no2:.2f}")
print(f"Schiefe:           {skew_no2:.2f}")

In [ ]:
sns.histplot(df["No2"], bins=100)

In [ ]:
# NO2 >= 300 oder == 500
df.loc[df["No2"] >= 300, "No2"] = np.nan

### Bereinigung der NO₂-Werte
Die NO₂-Werte zeigen eine stark rechtsschiefe Verteilung, mit einem Mittelwert von 10.55 µg/m³, einem Median von 8.40 µg/m³ und einer Schiefe von 10.08. Die große Mehrheit der Messwerte liegt unterhalb von 120 µg/m³, während einzelne Ausreißer bis 425 µg/m³ reichen. 

Zusätzlich tritt eine auffällige Häufung bei 500 µg/m³ auf, was sehr wahrscheinlich auf technische Platzhalterwerte zurückzuführen ist.
Um die Analyse nicht durch extreme Einzelwerte zu verzerren, wurden alle NO₂-Werte ab 300 µg/m³ ausgeschlossen.

In [ ]:
df["No2"].describe()

## Analyse und Bereinigung der So2-Werte

In [ ]:
sns.stripplot(x=df["So2"], size=2, jitter=0.3)
plt.title("SO₂ Einzelwerte")
plt.xlabel("SO₂-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
mean_so2 = df["So2"].mean()
median_so2 = df["So2"].median()
skew_so2 = skew(df["So2"].dropna())

print(f"Mittelwert (mean): {mean_so2:.2f}")
print(f"Median:            {median_so2:.2f}")
print(f"Schiefe:           {skew_so2:.2f}")

In [ ]:
# SO2 >= 300 oder == 500
df.loc[df["So2"] >= 300, "So2"] = np.nan

### Bereinigung der SO₂-Werte
Die Verteilung der Schwefeldioxid-Werte ist stark rechtsschief, mit einem Mittelwert von 4.10 µg/m³, einem Median von 2.60 µg/m³ und einer Schiefe von 10.45. Die große Mehrheit der Messwerte liegt unterhalb von 80 µg/m³, darüber hinaus treten vereinzelt höhere Werte auf – einige davon sind durchaus plausibel.

Auffällig ist jedoch eine Datenlücke im Bereich von 300 bis 400 µg/m³, gefolgt von insgesamt neun vereinzelten Extremwerten oberhalb von 400, von denen zwei exakt bei 500 µg/m³ liegen. Diese Muster deuten auf technisch bedingte Ausreißer oder Platzhalterwerte hin.
Um die Aussagekraft der Analyse zu erhöhen, wurden daher alle SO₂-Werte ab 300 µg/m³ ausgeschlossen.

## Analyse und Bereinigung der O3-Werte

In [ ]:
sns.stripplot(x=df["O3"], size=2, jitter=0.3)
plt.title("O3 Einzelwerte")
plt.xlabel("O3-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
mean_o3 = df["O3"].mean()
median_o3 = df["O3"].median()
skew_o3 = skew(df["O3"].dropna())

print(f"Mittelwert (mean): {mean_o3:.2f}")
print(f"Median:            {median_o3:.2f}")
print(f"Schiefe:           {skew_o3:.2f}")

In [ ]:
sns.histplot(df["O3"], bins=100)

In [ ]:
# O3 >= 380 oder == 500
df.loc[df["O3"] >= 380, "O3"] = np.nan

### Bereinigung der Ozon-Werte (O₃)
Die Verteilung der Ozonwerte zeigt eine deutliche Rechtsschiefe mit einem Mittelwert von 20.01 µg/m³, einem Median von 19.10 µg/m³ und einer Schiefe von 7.71. Die große Mehrheit der Messwerte liegt unterhalb von 100 µg/m³, während vereinzelte Extremwerte bis ca. 320 µg/m³ auftreten, die jedoch noch plausibel erscheinen.

Auffällig ist eine systematische Häufung im Bereich zwischen 380 und 390 µg/m³, die auf eine technisch bedingte Obergrenze oder einen Messfehler hinweist. Zusätzlich treten vereinzelte Werte beim Maximalwert von 500 µg/m³ auf, was stark auf einen Platzhalterwert schließen lässt.

Um die Analyse nicht durch solche systematischen Ausreißer zu verzerren, wurden daher alle O₃-Werte ab 380 µg/m³ aus dem Datensatz ausgeschlossen.

## Analyse und Bereinigung der Pm10-Werte

In [ ]:
sns.stripplot(x=df["Pm10"], size=2, jitter=0.3)
plt.title("Pm10 Einzelwerte")
plt.xlabel("Pm10-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
df["Pm10"].describe()

In [ ]:
mean_pm10 = df["Pm10"].mean()
median_pm10 = df["Pm10"].median()
skew_pm10 = skew(df["Pm10"].dropna())

print(f"Mittelwert (mean): {mean_pm10:.2f}")
print(f"Median:            {median_pm10:.2f}")
print(f"Schiefe:           {skew_pm10:.2f}")

In [ ]:
sns.histplot(df["O3"], bins=100)

ACHTUNG: Der Cutoff bei PM10 ist nicht so eindeutig zu setzen wie bei den anderen Schadstoffen. Es sind also zusätzliche Berechnungen sinnvoll.

In [ ]:
# Wo beginnt die Häufung bei der breiten Linie im Ausreißerbereich (ca. 820-840)?

df["Pm10"].value_counts().loc[lambda x: x.index > 820].sort_index()


In [ ]:
# "Datenmauer" plotten (nicht wirklich nötig, aber "Nice to have" für Leute, die gerne Visualisierungen mögen...)

high_values = df["Pm10"][df["Pm10"] > 800]
sns.histplot(high_values, binwidth=1)
plt.title("Häufigkeit der PM10-Werte > 800")
plt.xlabel("PM10-Wert (µg/m³)")
plt.ylabel("Anzahl")
plt.grid(True)
plt.show()


In [ ]:
# PM10 >= 867
df.loc[df["Pm10"] >= 867, "Pm10"] = np.nan


### Bereinigung der PM10-Werte
Die PM10-Werte zeigen eine deutlich rechtsschiefe Verteilung mit einem Mittelwert von 32.43 µg/m³, einem Median von 24.00 µg/m³ und einer Schiefe von 7.17. Der größte Teil der Messwerte liegt unterhalb von 200 µg/m³, mit einer kontinuierlich abnehmenden Dichte bis in den Bereich von ca. 850 µg/m³.

Eine detaillierte Analyse der Einzelwertverteilung ergab jedoch eine auffällige Häufung von Messwerten im Bereich 867–882 µg/m³, insbesondere 40 Messungen mit dem identischen Wert 867. Diese systematische Häufung spricht stark für einen technisch bedingten Fehler oder künstlich begrenzte Wertebereiche. Zusätzlich tritt der Platzhalterwert 999 33 Mal auf.

Um die Analyse nicht durch diese systematischen Verzerrungen zu beeinflussen, wurden alle PM10-Werte ab 867 µg/m³ aus dem Datensatz ausgeschlossen.

### Methode zum Entfernen von Platzhalterwerten und starken Ausreißern
Zur Datenbereinigung wurden extreme Ausreißer in den Schadstoffwerten nicht durch das Entfernen ganzer Zeilen, sondern durch das gezielte Ersetzen der jeweiligen Zellen mit NaN behandelt. So bleiben andere valide Messwerte in der gleichen Zeile erhalten.

In [ ]:
overview(df)

# Analyse und Bereinigung der Wetterdaten
Analog zum Vorgehen bei den Schadstoffen werden im Folgenden alle Wettervariablen untersucht und ggf. Platzhalterwerte und starke Ausreißer durch NaN ersetzt.

Anmerkung: Man KÖNNTE auch nur die Platzhalterwerte durch NaN ersetzten und die Ausreißer gesondert kennzeichen. Aber darauf wird jetzt erst einmal verzichtet.

In [ ]:
df[["Tavg", "Tmin", "Tmax", "Humidity", "Wspd", "Wdir", "Pres", "Prcp", "Dew"]].describe()


## Auswertung von describe()

Spalte    |         Bedingung für NaN              |                     Begründung (Tabelle noch formatieren!)

Tmax	    Tmax > 60	                                        weit über weltweit beobachtete Temperaturen

Humidity	Humidity < 0 oder > 100	                            physikalisch unmöglich / Platzhalter

Pres	    Pres > 1100	                                        ungewöhnlich hoch → Messfehler möglich

Wspd	    erst visuell prüfen, dann ggf. > 150	            Hurrikan-Grenze

Prcp	    nicht filtern, sondern visuell prüfen	            rechtsschiefe, aber plausible Naturverteilung

Dew	        ggf. ab –70 prüfen, aber tendenziell realistisch	vorsichtiger Umgang

In [ ]:
# Tmax

sns.stripplot(x=df["Tmax"], size=2, jitter=0.3)
plt.title("Tmax Einzelwerte")
plt.xlabel("Maximale Tages-Temperatur (°C)")
plt.grid(True)
plt.show()


In [ ]:
# Schiefe von Tmax
skew_Tmax = skew(df["Tmax"].dropna())
print(f"Schiefe:           {skew_Tmax:.2f}")

In [ ]:
# Humidity

sns.stripplot(x=df["Humidity"], size=2, jitter=0.3)
plt.title("Humidity Einzelwerte")
plt.xlabel("Maximale Feuchtigkeit")
plt.grid(True)
plt.show()


In [ ]:
# Schiefe von Humidity
skew_humid = skew(df["Humidity"].dropna())
print(f"Schiefe:           {skew_humid:.2f}")

In [ ]:
# Pressure

sns.stripplot(x=df["Pres"], size=2, jitter=0.3)
plt.title("Luftdruck Einzelwerte")
plt.xlabel("Maximaler Luftdruck")
plt.grid(True)
plt.show()

In [ ]:
# Schiefe von Pres
skew_pres = skew(df["Pres"].dropna())
print(f"Schiefe:           {skew_pres:.2f}")

In [ ]:
# Tmax: realistische Obergrenze bei 60°C
df.loc[df["Tmax"] > 60, "Tmax"] = np.nan

# Humidity: alles < 0 oder > 100 ist physikalisch nicht möglich
df.loc[(df["Humidity"] < 0) | (df["Humidity"] > 100), "Humidity"] = np.nan

# Pres: seltene Ausreißer über 1100 hPa → eher technisches Problem
df.loc[df["Pres"] > 1100, "Pres"] = np.nan

Extremwerte in den Wetterdaten wurden bereinigt:

    Tmax > 60 °C (Temperatursensorfehler)
    Humidity < 0 oder > 100 % (physikalisch unmöglich bzw. Platzhalter wie 999)
    Pres > 1100 hPa (sehr unwahrscheinlicher Luftdruck)
    Diese Werte wurden durch NaN ersetzt, um die Datenqualität zu verbessern.

Für die Variablen Tmax, Humidity und Pres wurden mithilfe von Stripplots und Schiefe-Analyse potenziell fehlerhafte Messwerte identifiziert.

Werte über 60 °C (Tmax), über 100 % Luftfeuchtigkeit (Humidity) und über 1100 hPa Luftdruck (Pres) wurden als unplausibel oder technisch bedingt eingestuft und durch NaN ersetzt, um die Datenqualität für nachfolgende Analysen zu sichern.

## Analyse und Bereinigung der Daten zum Niederschlag (Prcp) und zur Windgeschwindigkeit (Wspd)

In [ ]:
sns.stripplot(x=df["Prcp"], size=2, jitter=0.3)
plt.title("Prcp Einzelwerte")
plt.xlabel("Tagesniederschlag (mm)")
plt.grid(True)
plt.show()

In [ ]:
# Schiefe von Prcp
skew_prcp = skew(df["Prcp"].dropna())
print(f"Schiefe:           {skew_prcp:.2f}")

In [ ]:
sns.stripplot(x=df["Wspd"], size=2, jitter=0.3)
plt.title("Wspd Einzelwerte")
plt.xlabel("Windgeschwindigkeit (km/h)")
plt.grid(True)
plt.show()

In [ ]:
# Schiefe von Wspd
skew_wspd = skew(df["Wspd"].dropna())
print(f"Schiefe:           {skew_wspd:.2f}")

In [ ]:
# vorsichtige Ausreißerentfernung (Werte nicht komplett unrealsitisch, aber für Modelle evtl. störend)

df.loc[df["Prcp"] > 400, "Prcp"] = np.nan

df.loc[df["Wspd"] > 80, "Wspd"] = np.nan

Die Verteilungen von Niederschlagsmenge (Prcp) und Windgeschwindigkeit (Wspd) zeigen typische rechtsschiefe Muster mit wenigen extrem hohen Einzelwerten.

    Für Prcp wurden Werte über 400 mm als Ausreißer interpretiert und durch NaN ersetzt.
    Für Wspd wurde eine obere Grenze von 80 km/h festgelegt, da darüber nur vereinzelte, isolierte Extremwerte vorhanden waren.
    Beide Filter sollen die Datenqualität verbessern, ohne relevante meteorologische Extremereignisse auszusortieren.

# Welche Features könnten zusammenhängen (Kollinearität)?

In [ ]:
# Collinearity of features? (heatmap)

# > 0.8 = strong correlation
# 0.5 - 0.8 = moderate correlation
# < 0.5 = weak correlation

# calculate correlation matrix (Pearson)
corr_matrix = df.select_dtypes(include=['number']).corr()

# Display heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Feature correlations (Pearson)");


In [ ]:
# Show only strong and moderate correlations (>|0.5|); leave out main diagonal (1.0)

# Calculte matrix
corr_matrix = df.select_dtypes(include=['number']).corr()

# extract only strong and moderate correlations (>|0.5|); leave out main diagonal (1.0)
strong_corrs = corr_matrix[(corr_matrix.abs() > 0.4) & (corr_matrix.abs() < 1.0)]

# Convert df to long list (.stack) and reset index
strong_corrs = strong_corrs.stack().reset_index()
strong_corrs.columns = ["Feature 1", "Feature 2", "Korrelation"]

# remove redundant rows (note: the "<"-sign here refers to alphabetic order of feature names, not to numbers of any kind!)
strong_corrs = strong_corrs.loc[strong_corrs["Feature 1"] < strong_corrs["Feature 2"]]

strong_corrs

In [ ]:
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Feature Correlations (Pearson)");


## Analyse der Korrelationen (Pearson) und Vorschläge für weitere Analysen

Die Korrelationsmatrix zeigt mehrere starke und inhaltlich gut erklärbare Zusammenhänge zwischen den numerischen Variablen. Besonders deutlich sind folgende Muster:

    Feinstaub- und Stickstoffdioxidwerte hängen zusammen:
    Es besteht eine starke Korrelation zwischen PM10 und PM2.5 (r = 0.83), was plausibel ist, da PM2.5 eine Teilmenge von PM10 ist. Zusätzlich korreliert NO₂ moderat mit beiden Feinstaubkomponenten (r ≈ 0.42–0.48), was auf gemeinsame Emissionsquellen wie Verkehr oder Industrie hindeutet.

    Temperaturvariablen sind stark untereinander korreliert:
    Die Tagesmitteltemperatur (Tavg) steht in sehr engem Zusammenhang mit Tmin und Tmax (r ≈ 0.97). Auch Tmin und Tmax selbst sind hoch korreliert (r = 0.90). Das ist mathematisch und physikalisch naheliegend und spricht dafür, nicht alle drei Variablen gleichzeitig zu verwenden, um Redundanz zu vermeiden.

    Der Taupunkt (Dew) korreliert stark mit Temperatur:
    Die stärkste Korrelation liegt zwischen Dew und Tmin (r = 0.87), gefolgt von Tavg (r = 0.82). Dies spiegelt wider, dass die Luftfeuchtigkeit – und damit der Taupunkt – eng mit der Umgebungstemperatur zusammenhängt.

    Einige schwächer negative Korrelationen deuten auf atmosphärische Zusammenhänge hin:
    Der Luftdruck (Pres) korreliert moderat negativ mit Dew (r = –0.43) und Tmin (r = –0.42), was mit typischen meteorologischen Prozessen in Zusammenhang stehen kann (z. B. feuchtwarme Luft in Tiefdruckgebieten).

Diese Ergebnisse helfen dabei, hoch korrelierte bzw. redundante Variablen zu erkennen und gezielt für weitere Analysen (z. B. Clusteranalyse oder Modellierung) geeignete Features auszuwählen.

Für viele Verfahren, wie z. B. Clustering oder Regressionsmodelle, ist es ratsam, von stark korrelierten Variablen jeweils nur eine zu verwenden, um Verzerrungen oder sogenannte Multikollinearität zu vermeiden.

Alternativ können Hauptkomponentenanalyse (PCA) oder andere dimensionalitätsreduzierende Verfahren genutzt werden, um mehrere stark korrelierte Variablen zu einer gemeinsamen Komponente zusammenzufassen, ohne wesentliche Information zu verlieren.


In [ ]:
# Plot changes throughout the year. Example: Analyse CO concentration per month

# calculate mean CO per month
monthly_co = df.groupby("Month")["Co"].mean()
# monthly_co = df.groupby("Month")["Co"].mean()

# create plot
plt.figure(figsize=(10, 5))
plt.plot(monthly_co.index, monthly_co.values, marker='o', linestyle='-')
plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Durchschnittliche CO-Konzentration pro Monat")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.grid(True);

In [ ]:
# Trennung der Daten in Nord- und Südhalbkugel

# Definieren, welche Länder zur Nord- und Südhalbkugel gehören
northern_hemisphere_countries = {
    "US", "CA", "MX", "DE", "FR", "GB", "RU", "CN", "JP", "IN", "IT", "ES", "PL", "TR", "IR", "KR", "UA", "NL", "BE",
    "CH", "SE", "AT", "NO", "FI", "DK", "GR", "CZ", "HU", "RO", "BG", "PT", "IE", "SK", "HR", "LT", "SI", "LV", "EE"
}
southern_hemisphere_countries = {
    "AU", "NZ", "AR", "BR", "ZA", "CL", "ID", "PE", "BO", "EC", "PY", "UY", "MG"
}

# Daten für Nord- und Südhalbkugel filtern
df_north = df[df["Country"].isin(northern_hemisphere_countries)]
df_south = df[df["Country"].isin(southern_hemisphere_countries)]

# Mittlere CO-Konzentration pro Monat berechnen
monthly_co_north = df_north.groupby("Month")["Co"].mean()
monthly_co_south = df_south.groupby("Month")["Co"].mean()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_co_north.index, monthly_co_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_co_south.index, monthly_co_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Vergleich der CO-Konzentration auf Nord- und Südhalbkugel")
plt.xticks(range(1, 13))  # Monatsskala 1-12
plt.legend()
plt.grid(True);

In [ ]:
# CO-Werte über das Jahr inklusive Streuung

plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x="Month", y="Co", errorbar="sd", marker="o")
plt.xlabel("Monat")
plt.ylabel("CO-Wert")
plt.title("Kohlenmonoxidwerte Im Jahresverlauf")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.show()

# Die Grafik ist nicht intuitiv aussagekräftig. Die Streuung ist zu stark.

In [ ]:
# Streuung mit Achsenbegrenzung

# Gruppiere nach Monat
grouped = df.groupby("Month")["Co"]
mean = grouped.mean()
std = grouped.std()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(mean.index, mean.values, marker='o', linestyle='-', label='Mittelwert CO')
plt.fill_between(mean.index, mean - std, mean + std, color='blue', alpha=0.2, label='±1 SD')

# Achsen, Titel, Beschriftungen
plt.xlabel("Monat")
plt.ylabel("CO-Konzentration (ppm)")
plt.title("Durchschnittliche CO-Konzentration mit Streuung")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])

# Fokus auf relevanten y-Bereich
plt.ylim(3, 7)  # oder anpassen je nach Datensatz

plt.grid(True)
plt.legend()
plt.show()

# Ergebnis: Die Streuung bedeckt die gesamte Diagrammfläche. Es sollten als die Städte, die die größte Verzerrung bewirken, aussortiert werden.


In [ ]:
co_by_city = df.groupby("City")["Co"].mean().sort_values(ascending=False)
print(co_by_city.head(10))

# Ashkelon ist ein unrealistischer Ausreißer. Siehe separates NB "Ashkelon": Es gibt nur Messwerte für Jan und Feb 2022, und diese schwanken extrem. Vielleicht kaputtes Messgerät.

In [ ]:
# calculate std per city

co_std_by_city = df.groupby("City")["Co"].std().sort_values(ascending=False)
print(co_std_by_city.head(10))

In [ ]:
# combine mean and std to determine cities that cause the bigges distortions.

co_stats = df.groupby("City")["Co"].agg(["mean", "std"]).sort_values(by="mean", ascending=False)
print(co_stats.head(10))

In [ ]:
# The three Israeli cities of Ashkelon, Ashdod and Netanya show highly volatile CO values, measured only in certain months.
# This distorts the graph to an extent that calls for dropping these three cities from further analyes of CO.

In [ ]:
# Drop distorting cities from fürther analyses
exclude_cities = ["Ashkelon", "Ashdod", "Netanya"]
df_cleaned = df[~df["City"].isin(exclude_cities)]

In [ ]:
# Streuung mit Achsenbegrenzung, ohne Ausreißerstädte in Israel

# Gruppiere nach Monat
grouped = df_cleaned.groupby("Month")["Co"]
mean = grouped.mean()
std = grouped.std()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(mean.index, mean.values, marker='o', linestyle='-', label='Mittelwert CO')
plt.fill_between(mean.index, mean - std, mean + std, color='blue', alpha=0.2, label='±1 SD')

# Achsen, Titel, Beschriftungen
plt.xlabel("Monat")
plt.ylabel("CO-Konzentration (ppm)")
plt.title("Durchschnittliche CO-Konzentration mit Streuung")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])

# Fokus auf relevanten y-Bereich
plt.ylim(3, 7)  # oder anpassen je nach Datensatz

plt.grid(True)
plt.legend()
plt.show()

# Ergebnis: Die Streuung bedeckt die gesamte Diagrammfläche. Es sollten als die Städte, die die größte Verzerrung bewirken, aussortiert werden.


In [ ]:
# CO-Werte über das Jahr inklusive Streuung

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_cleaned, x="Month", y="Co", errorbar="sd", marker="o")
plt.xlabel("Monat")
plt.ylabel("CO-Wert")
plt.title("Kohlenmonoxidwerte Im Jahresverlauf")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.show()

# Die Grafik ist immer noch nicht nicht intuitiv aussagekräftig. Die Streuung ist weiterhin zu stark.

In [ ]:
# Ursachenforschung für die starke CO-Streuung

co_std_cleaned = df_cleaned.groupby("City")["Co"].std().sort_values(ascending=False)
print(co_std_cleaned.head(10))

In [ ]:
co_stats_cleaned = df_cleaned.groupby("City")["Co"].agg(["mean", "std"]).sort_values(by="std", ascending=False)
print(co_stats_cleaned.head(10))

In [ ]:
# Weitere Städte mit extremer Varianz entfernen, um eine übersichtliche globale Darstellung zu erzielen

more_extreme = ["Portland", "Mérida", "Zamboanga", "Butuan", "Hạ long", "Oaxaca", "Isfahan", "San luis potosí", "Tabriz", "Tallahassee"]
exclude_cities = ["Ashkelon", "Ashdod", "Netanya"] + more_extreme
df_cleaned2 = df[~df["City"].isin(exclude_cities)]

In [ ]:
# CO-Werte über das Jahr inklusive Streuung, weiter reduziert

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_cleaned2, x="Month", y="Co", marker="o")
plt.fill_between(mean.index, mean - std * 0.5, mean + std * 0.5, alpha=0.2, label='±0.5 SD')
plt.xlabel("Monat")
plt.ylabel("CO-Wert")
plt.title("Kohlenmonoxidwerte Im Jahresverlauf")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.show()

# Die Grafik ist immer noch nicht nicht intuitiv aussagekräftig. Die Streuung ist weiterhin zu stark.

In [ ]:

# Nord- und Südhalbkugel anhand des Features "Latitude" trennen

df_cleaned2.loc[:, "Hemisphere"] = df_cleaned2.loc[:, "Latitude"].apply(lambda x: "Nordhalbkugel" if x >= 0 else "Südhalbkugel")



In [ ]:
df_north = df_cleaned2[df_cleaned2["Hemisphere"] == "Nordhalbkugel"]
df_south = df_cleaned2[df_cleaned2["Hemisphere"] == "Südhalbkugel"]

In [ ]:
# Nordhalbkugel
grouped_north = df_north.groupby("Month")["Co"]
mean_north = grouped_north.mean()
std_north = grouped_north.std()

# Südhalbkugel
grouped_south = df_south.groupby("Month")["Co"]
mean_south = grouped_south.mean()
std_south = grouped_south.std()

# Plot
plt.figure(figsize=(10, 5))

plt.plot(mean_north.index, mean_north, marker='o', label="Nordhalbkugel")
plt.fill_between(mean_north.index, mean_north - std_north, mean_north + std_north, alpha=0.2)

plt.plot(mean_south.index, mean_south, marker='s', label="Südhalbkugel")
plt.fill_between(mean_south.index, mean_south - std_south, mean_south + std_south, alpha=0.2)

plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.xlabel("Monat")
plt.ylabel("CO-Konzentration (ppm)")
plt.title("Saisonale CO-Muster nach Hemisphäre")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Mittlere CO-Konzentration pro Monat berechnen
monthly_co_north = df_north.groupby("Month")["Co"].mean()
monthly_co_south = df_south.groupby("Month")["Co"].mean()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_co_north.index, monthly_co_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_co_south.index, monthly_co_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Vergleich der CO-Konzentration auf Nord- und Südhalbkugel")
plt.xticks(range(1, 13))  # Monatsskala 1-12
plt.legend()
plt.grid(True);

In [ ]:
# Mittlere CO-Konzentration pro Monat berechnen- Median statt Mean, weil robuster gegen Ausreißer
monthly_co_north = df_north.groupby("Month")["Co"].median()
monthly_co_south = df_south.groupby("Month")["Co"].median()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_co_north.index, monthly_co_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_co_south.index, monthly_co_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Vergleich der CO-Konzentration auf Nord- und Südhalbkugel")
plt.xticks(range(1, 13))  # Monatsskala 1-12
plt.legend()
plt.grid(True);

In [ ]:
# sns.histplot(df_north["Co"], kde=True)
# sns.histplot(df_south["Co"], kde=True)

plt.figure(figsize=(10, 5))

# Nordhalbkugel
sns.histplot(df_north["Co"], kde=True, label="Nordhalbkugel", stat="density", element="step", fill=True)

# Südhalbkugel
sns.histplot(df_south["Co"], kde=True, label="Südhalbkugel", stat="density", element="step", fill=True)

# Achsen & Legende
plt.xlabel("CO-Konzentration (ppm)")
plt.ylabel("Dichte")
plt.title("Verteilung der CO-Werte nach Hemisphäre")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))

# Histogramm Nordhalbkugel
sns.histplot(df_north["Co"], kde=True, stat="density", label="Nordhalbkugel", color="steelblue", fill=True)

# Histogramm Südhalbkugel
sns.histplot(df_south["Co"], kde=True, stat="density", label="Südhalbkugel", color="darkorange", fill=True)

# Zoom auf interessanten Bereich
plt.xlim(0, 20)    # X-Achse (CO-Werte) begrenzen
plt.ylim(0, 0.7)   # Y-Achse (Dichte) begrenzen

# Titel & Achsen
plt.xlabel("CO-Konzentration (ppm)")
plt.ylabel("Dichte")
plt.title("CO-Verteilung nach Hemisphäre (vergrößerter Bereich)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Mittlere CO-Konzentration pro Land berechnen
country_co_avg = df.groupby("Country")["Co"].mean().sort_values(ascending=False)

# Barplot erstellen
plt.figure(figsize=(12, 6))
country_co_avg.plot(kind='bar', color='b', alpha=0.7)
plt.xlabel("Land")
plt.ylabel("Mittlere CO-Konzentration")
plt.title("Durchschnittliche CO-Konzentration pro Land")
plt.xticks(rotation=90)  # Länderbeschriftung drehen für bessere Lesbarkeit
plt.grid(axis='y', linestyle='--', alpha=0.7);


In [ ]:
# Mittlere CO-Konzentration pro Land berechnen

# Mindestanzahl an CO-Messwerten pro Land, um in die Analyse aufgenommen zu werden
min_measurements = 100  # Falls nötig, anpassen

# Anzahl der CO-Messwerte pro Land berechnen
country_co_counts = df.groupby("Country")["Co"].count()

# Nur Länder behalten, die mindestens `min_measurements` Messwerte haben
valid_countries = country_co_counts[country_co_counts >= min_measurements].index

# Gefilterten DataFrame mit diesen Ländern erstellen
df_valid_countries = df[df["Country"].isin(valid_countries)]

# Mittlere CO-Konzentration für diese Länder berechnen
country_co_avg_filtered = df_valid_countries.groupby("Country")["Co"].mean().sort_values(ascending=False)

# Falls nach der Filterung noch Daten vorhanden sind, Plot erstellen
if not country_co_avg_filtered.empty:
    plt.figure(figsize=(12, 6))
    country_co_avg_filtered.plot(kind='bar', color='b', alpha=0.7)
    plt.xlabel("Land")
    plt.ylabel("Mittlere CO-Konzentration")
    plt.title("Durchschnittliche CO-Konzentration pro Land (nur Länder mit ausreichenden Messwerten)")
    plt.xticks(rotation=90)  # Länderbeschriftung drehen für bessere Lesbarkeit
    plt.grid(axis='y', linestyle='--', alpha=0.7)
else:
    print("Keine ausreichenden Daten für CO-Werte in den Ländern verfügbar.")


In [ ]:
country_co_avg_filtered.head(20).plot(kind='bar')


In [ ]:
# Vergleich von Schadstoffen in verschiedenen Städten

# Liste der relevanten Schadstoff-Spalten (falls sie in den Daten vorhanden sind)
pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]

# DataFrame mit nur den relevanten Spalten (fehlende Werte entfernen)
df_pollutants = df[pollutants].dropna()

# Korrelationsmatrix berechnen
corr_matrix = df_pollutants.corr()

# Heatmap der Korrelationen erstellen
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Korrelation zwischen CO und anderen Schadstoffen");


In [ ]:
df[["Co", "No2"]].corr()

In [ ]:
# Korrelation Schadstoffe und Wettervariablen

# Liste der Schadstoffe und Wettervariablen
pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]
weather_vars = ["temperature", "pressure", "humidity", "dew", "wind-speed", "wind-gust"]

# DataFrame mit nur den relevanten Spalten (fehlende Werte entfernen)
df_pollutants_weather = df[pollutants + weather_vars].dropna()

# Korrelationsmatrix berechnen
corr_matrix_weather = df_pollutants_weather.corr()

# Heatmap der Korrelationen zwischen Schadstoffen & Wettervariablen
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix_weather, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Korrelation zwischen Schadstoffen und Wetterfaktoren");

In [ ]:
df[["co", "temperature"]].corr()

In [ ]:
# Schadstoffbelastung über die Zeit in verschiedenen Städten

# Durchschnittliche Schadstoffwerte pro Jahr berechnen
pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]
yearly_trends = df.groupby("Year")[pollutants].mean()

# Liniendiagramm für langfristige Trends erstellen
plt.figure(figsize=(12, 6))
for pollutant in pollutants:
    plt.plot(yearly_trends.index, yearly_trends[pollutant], marker='o', linestyle='-', label=pollutant)

plt.xlabel("Jahr")
plt.ylabel("Mittlere Konzentration")
plt.title("Langfristige Entwicklung der Schadstoffwerte")
plt.legend()
plt.grid(True);


In [ ]:
# Ohne 2014 und 2025, weil zu wenige Daten

# Schadstoffe, die analysiert werden sollen
pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]

# Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
df["year"] = pd.to_numeric(df["Year"], errors="coerce")

# Durchschnittliche Schadstoffwerte pro Jahr berechnen, aber 2014 & 2025 ausschließen
yearly_trends = df.groupby("Year")[pollutants].mean()
yearly_trends = yearly_trends.loc[(yearly_trends.index > 2014) & (yearly_trends.index < 2025)]

# Liniendiagramm für langfristige Trends erstellen
plt.figure(figsize=(12, 6))
for pollutant in pollutants:
    plt.plot(yearly_trends.index, yearly_trends[pollutant], marker='o', linestyle='-', label=pollutant)

plt.xlabel("Jahr")
plt.ylabel("Mittlere Konzentration")
plt.title("Langfristige Entwicklung der Schadstoffwerte (ohne 2014 & 2025)")
plt.legend()
plt.grid(True);


In [ ]:
num_cities_with_data = df.loc[:, ["City", "Co", "No2", "So2", "O3", "Pm25", "Pm10"]].dropna(subset=["Co", "No2", "So2", "O3", "Pm25", "Pm10"], how="all")["City"].nunique()
print(f"Anzahl der Städte mit mindestens einem Messwert: {num_cities_with_data}")

In [ ]:
    import matplotlib.pyplot as plt

    # Schadstoffe, die analysiert werden sollen
    pollutants = ["Co", "No2", "O3", "So2", "Pm10", "Pm25"]

    # Länder nach Regionen gruppieren
    regions = {
        "Europe": {"DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE"},
        "North America": {"US", "CA", "MX"},
        "South America": {"BR", "AR", "CO", "CL", "PE"},
        "Asia": {"CN", "IN", "JP", "KR", "IR"},
        "Africa": {"ZA", "EG", "NG"},
        "Oceania": {"AU", "NZ"}
    }

    # Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

    # Regionen durchgehen & Diagramme erstellen
    fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharex=True, sharey=True)  # Einheitliche Achsen für bessere Vergleichbarkeit
    axes = axes.flatten()  # 2D-Array in 1D-Array umwandeln

    # Speichert alle Linien für die gemeinsame Legende
    handles, labels = [], []

    for i, (region, countries) in enumerate(regions.items()):
        df_region = df[df["Country"].isin(countries)]
        yearly_trends_region = df_region.groupby("Year")[pollutants].mean()
        yearly_trends_region = yearly_trends_region.loc[(yearly_trends_region.index > 2014) & (yearly_trends_region.index < 2025)]
        
        ax = axes[i]
        for pollutant in pollutants:
            line, = ax.plot(yearly_trends_region.index, yearly_trends_region[pollutant], marker='o', linestyle='-', label=pollutant)
            
            # Speichert eine Linie pro Schadstoff für die gemeinsame Legende
            if i == 0:  
                handles.append(line)
                labels.append(pollutant)

        ax.set_title(region)
        ax.set_xlabel("Jahr")
        ax.set_ylabel("Mittelwert")
        ax.grid(True)

    for ax in axes[:3]:  # Die ersten drei Subplots sind in der oberen Reihe
        ax.xaxis.set_tick_params(labelbottom=True)

    # Gemeinsame Legende unterhalb der Subplots platzieren
    fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(pollutants))

    # Layout optimieren
    plt.tight_layout();

In [ ]:
# Sechs Top-Länder in Europa

import matplotlib.pyplot as plt

# Schadstoffe, die analysiert werden sollen
pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

# Europäische Länder definieren
european_countries = {"DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE"}

# Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Nur europäische Länder auswählen
df_europe = df[df["Country"].isin(european_countries)]

# Länder mit den meisten Messwerten identifizieren
top_countries = df_europe["Country"].value_counts().nlargest(6).index  # Falls nur 6 Länder visualisiert werden sollen

# Subplots für die gewählten Länder erstellen
fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharex=True, sharey=True)  # 2 Reihen, 3 Spalten
axes = axes.flatten()

for i, country in enumerate(top_countries):
    df_country = df_europe[df_europe["Country"] == country]
    yearly_trends_country = df_country.groupby("year")[pollutants].mean()
    yearly_trends_country = yearly_trends_country.loc[(yearly_trends_country.index > 2014) & (yearly_trends_country.index < 2025)]
    
    ax = axes[i]
    for pollutant in pollutants:
        ax.plot(yearly_trends_country.index, yearly_trends_country[pollutant], marker='o', linestyle='-', label=pollutant)
    
    ax.set_title(country)
    ax.set_xlabel("Jahr")
    ax.set_ylabel("Mittelwert")
    ax.grid(True)

# Gemeinsame Legende unterhalb der Subplots platzieren
fig.legend(pollutants, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(pollutants))

# Layout optimieren
plt.tight_layout();

In [ ]:
# Korrekation zwischen Ozon und anderen Faktoren

# Relevante Spalten auswählen
pollutants = ["co", "no2", "so2", "pm10", "pm25", "o3"]
weather_vars = ["temperature", "pressure", "humidity", "dew", "wind-speed", "wind-gust"]

# DataFrame mit nur den relevanten Spalten (fehlende Werte entfernen)
df_ozone_corr = df[pollutants + weather_vars].dropna()

# Korrelationsmatrix berechnen
corr_matrix_ozone = df_ozone_corr.corr()

# Heatmap der Korrelationen zwischen Ozon und anderen Faktoren
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix_ozone, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Korrelation zwischen Ozon (O₃) und anderen Faktoren");


In [ ]:
# Durchschnittliche Ozonwerte pro Monat berechnen
monthly_o3 = df.groupby("month")["o3"].mean()

# Liniendiagramm für die saisonale Entwicklung von Ozon erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_o3.index, monthly_o3.values, marker='o', linestyle='-', color='b')
plt.xlabel("Monat")
plt.ylabel("Mittlere O₃-Konzentration")
plt.title("Saisonale Entwicklung der Ozonwerte (O₃)")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.grid(True);

In [ ]:
# Länder nach Hemisphäre aufteilen
northern_hemisphere_countries = {
    "US", "CA", "MX", "DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE", "RU", "CN", "JP", "IN", "KR"
}
southern_hemisphere_countries = {
    "AU", "NZ", "AR", "BR", "ZA", "CL", "ID", "PE", "BO", "EC", "PY", "UY", "MG"
}

# Daten für Nord- und Südhalbkugel filtern
df_north = df[df["Country"].isin(northern_hemisphere_countries)]
df_south = df[df["Country"].isin(southern_hemisphere_countries)]

# Durchschnittliche Ozonwerte pro Monat für beide Hemisphären berechnen
monthly_o3_north = df_north.groupby("month")["o3"].mean()
monthly_o3_south = df_south.groupby("month")["o3"].mean()

# Plot erstellen
plt.figure(figsize=(10, 5))
plt.plot(monthly_o3_north.index, monthly_o3_north.values, marker='o', linestyle='-', label="Nordhalbkugel", color='b')
plt.plot(monthly_o3_south.index, monthly_o3_south.values, marker='o', linestyle='-', label="Südhalbkugel", color='r')

plt.xlabel("Monat")
plt.ylabel("Mittlere O₃-Konzentration")
plt.title("Vergleich der Ozonwerte (O₃) zwischen Nord- und Südhalbkugel")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"])
plt.legend()
plt.grid(True);



In [ ]:
import matplotlib.pyplot as plt

# Europäische Länder definieren
european_countries = {"DE", "FR", "GB", "IT", "ES", "PL", "NL", "SE", "AT", "CH", "BE"}

# Falls "year" als String gespeichert ist, in numerischen Wert umwandeln
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Nur europäische Länder auswählen
df_europe = df[df["Country"].isin(european_countries)]

# Jahre 2014 & 2025 aus der Analyse entfernen
df_europe = df_europe[(df_europe["year"] > 2014) & (df_europe["year"] < 2025)]

# Nur Länder behalten, die tatsächlich O₃-Werte haben
countries_with_o3 = df_europe.groupby("Country")["o3"].count()
valid_countries = countries_with_o3[countries_with_o3 > 0].index  # Länder mit vorhandenen O₃-Werten

# DataFrame auf diese Länder filtern
df_europe = df_europe[df_europe["Country"].isin(valid_countries)]

# Standardabweichung (Schwankungsstärke) von O₃ pro Jahr & Land berechnen
ozone_volatility = df_europe.groupby(["year", "Country"])["o3"].std().unstack()

# Diagramm erstellen
fig, ax = plt.subplots(figsize=(12, 6))
handles = []  # Zum Speichern der Linien für die Legende
labels = []   # Zum Speichern der Ländernamen

for country in ozone_volatility.columns:
    line, = ax.plot(ozone_volatility.index, ozone_volatility[country], marker='o', linestyle='-', label=country)
    handles.append(line)
    labels.append(country)

ax.set_xlabel("Jahr")
ax.set_ylabel("Standardabweichung von O₃ (Schwankungsstärke)")
ax.set_title("Jährliche Schwankungsstärke der Ozonwerte in europäischen Ländern (nur Länder mit Daten)")
ax.grid(True)

# Legende unterhalb des Plots platzieren
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.15), ncol=len(valid_countries))

# Layout optimieren
plt.tight_layout();

In [ ]:
# Clusteranalyse zur Schadstoffbelastung

from sklearn.preprocessing import StandardScaler

# Liste der Schadstoff-Features für das Clustering
pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

# Durchschnittliche Schadstoffwerte pro Land berechnen
df_cluster = df.groupby("Country")[pollutants].mean().dropna()

# Daten skalieren (K-Means ist empfindlich gegenüber unterschiedlichen Skalen)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Ergebnis als DataFrame zurückgeben
df_cluster_scaled = pd.DataFrame(df_scaled, index=df_cluster.index, columns=pollutants)

# Überprüfen, ob die Daten korrekt vorbereitet sind
df_cluster_scaled.head()


In [ ]:
from sklearn.cluster import KMeans

# Teste verschiedene Clusterzahlen (k = 1 bis 10)
inertia = []
k_values = range(1, 31)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_cluster_scaled)
    inertia.append(kmeans.inertia_)  # Speichert den Fehler (Inertia)

# Elbow-Plot erstellen
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, marker='o', linestyle='-')
plt.xlabel("Anzahl der Cluster (k)")
plt.ylabel("Inertia (Fehler)")
plt.title("Elbow-Methode zur Bestimmung der optimalen Clusterzahl")
plt.grid(True);


In [ ]:
from sklearn.metrics import silhouette_score

# Teste verschiedene Clusterzahlen (k = 2 bis 10, weil k=1 keine Silhouettenwertung hat)
silhouette_scores = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(df_cluster_scaled)
    score = silhouette_score(df_cluster_scaled, labels)
    silhouette_scores.append(score)

# Silhouetten-Plot erstellen
plt.figure(figsize=(8, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o', linestyle='-')
plt.xlabel("Anzahl der Cluster (k)")
plt.ylabel("Silhouetten-Koeffizient")
plt.title("Silhouetten-Analyse zur Bestimmung der optimalen Clusterzahl")
plt.grid(True);


Entscheidung: 5 Cluster nehmen

In [ ]:
# K-Means Clustering mit k=5
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df_cluster_scaled["Cluster"] = kmeans.fit_predict(df_cluster_scaled)

# Cluster-Zugehörigkeit pro Land anzeigen
df_clusters = df_cluster_scaled[["Cluster"]]
df_clusters.head()


In [ ]:
df_clusters["Cluster"].value_counts()


In [ ]:
df_cluster_scaled.groupby("Cluster").mean()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.boxplot(x=df_cluster_scaled["Cluster"], y=df_cluster_scaled["pm25"])
plt.xlabel("Cluster")
plt.ylabel("PM2.5-Werte (standardisiert)")
plt.title("Vergleich der PM2.5-Werte in den Clustern")
plt.grid(True);

In [ ]:
# Länder und ihre Cluster-Zugehörigkeit anzeigen
df_clusters_sorted = df_clusters.sort_values(by="Cluster")  # Nach Cluster sortieren
df_clusters_sorted


In [ ]:
# Test mit 4 oder 6 Clustern

# K-Means mit 4 und 6 Clustern testen
for k in [4, 6]:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    df_cluster_scaled[f"Cluster_{k}"] = kmeans.fit_predict(df_cluster_scaled)

# Ergebnisse anzeigen
df_clusters_test = df_cluster_scaled[["Cluster_4", "Cluster_6"]]
df_clusters_test


In [ ]:
df_cluster_scaled.groupby("Cluster").mean()


In [ ]:
# K-Means nochmal ausführen, mit Stadt statt Land

from sklearn.preprocessing import StandardScaler

# Liste der Schadstoff-Features für das Clustering
pollutants = ["co", "no2", "o3", "so2", "pm10", "pm25"]

# Durchschnittliche Schadstoffwerte pro Stadt berechnen
df_cluster = df.groupby("City")[pollutants].mean().dropna()

# Daten skalieren (K-Means ist empfindlich gegenüber unterschiedlichen Skalen)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Ergebnis als DataFrame zurückgeben
df_cluster_scaled = pd.DataFrame(df_scaled, index=df_cluster.index, columns=pollutants)

# Überprüfen, ob die Daten korrekt vorbereitet sind
df_cluster_scaled.head()


In [ ]:

# Teheran rausschmeißen


from sklearn.cluster import KMeans

# Teste verschiedene Clusterzahlen (k = 1 bis 30) ohne Teheran
inertia_no_tehran = []
k_values = range(1, 31)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_cluster_no_tehran.drop(columns=["Cluster"]))  # Clustering ohne vorherige Cluster-Spalte
    inertia_no_tehran.append(kmeans.inertia_)

# Elbow-Plot erstellen
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia_no_tehran, marker='o', linestyle='-')
plt.xlabel("Anzahl der Cluster (k)")
plt.ylabel("Inertia (Fehler)")
plt.title("Elbow-Methode ohne Teheran")
plt.grid(True);



In [ ]:
# K-Means Clustering mit 4 Clustern (oder einer anderen optimalen Zahl)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_cluster_scaled["Cluster"] = kmeans.fit_predict(df_cluster_scaled)

# Cluster-Zugehörigkeit pro Stadt anzeigen
df_clusters = df_cluster_scaled[["Cluster"]]
df_clusters_sorted = df_clusters.sort_values(by="Cluster")  # Sortiert nach Cluster
df_clusters_sorted


In [ ]:
# Teheran aus dem DataFrame entfernen
df_cluster_no_tehran = df_cluster_scaled.drop(index="Tehran")

# K-Means erneut ausführen mit 5 Clustern (oder einer anderen optimalen Zahl)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df_cluster_no_tehran["Cluster"] = kmeans.fit_predict(df_cluster_no_tehran)

# Neue Cluster-Zugehörigkeit der Städte anzeigen
df_clusters_no_tehran = df_cluster_no_tehran[["Cluster"]]
df_clusters_no_tehran_sorted = df_clusters_no_tehran.sort_values(by="Cluster")  # Sortiert nach Cluster
df_clusters_no_tehran_sorted
